# Description

It combines all clustering solutions generated into a single consolidated solution using consensus clustering.

# Environment variables

In [1]:
from IPython.display import display

# set numpy n_jobs to 1, since I'll be using n_jobs later
NUMPY_N_JOBS = 1
display(NUMPY_N_JOBS)

1

In [2]:
%env MKL_NUM_THREADS=$NUMPY_N_JOBS
%env OPEN_BLAS_NUM_THREADS=$NUMPY_N_JOBS
%env NUMEXPR_NUM_THREADS=$NUMPY_N_JOBS
%env OMP_NUM_THREADS=$NUMPY_N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [5]:
np.random.seed(0)

In [6]:
RANDOM_STATES_ITER = iter(np.random.randint(0, np.iinfo(np.int32).max, size=100000))

In [7]:
# n_init parameter for DeltaSpectralClustering
# a high number should produce more stable final solutions
SC_N_INIT = 50

In [8]:
NULL_DIR = conf.RESULTS["CLUSTERING_NULL_DIR"] / "shuffle_genes"

## Consensus clustering

In [9]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 60

display(CLUSTERING_OPTIONS)

{'K_MIN': 2, 'K_MAX': 60}

In [10]:
# output dir for this notebook
RESULTS_DIR = Path(NULL_DIR, "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/opt/data/results/clustering/null_sims/shuffle_genes/consensus_clustering')

# Load ensemble

In [11]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/opt/data/results/clustering/null_sims/shuffle_genes/consensus_clustering/ensemble.npy')

In [12]:
full_ensemble = np.load(output_file)

In [13]:
display(full_ensemble.shape)

(3844, 3752)

# Load ensemble coassociation distance matrix

In [14]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/opt/data/results/clustering/null_sims/shuffle_genes/consensus_clustering/ensemble_coassoc_matrix.npy')

In [15]:
ensemble_coassoc_matrix = np.load(output_file)

In [16]:
display(ensemble_coassoc_matrix.shape)

(3752, 3752)

In [17]:
display(ensemble_coassoc_matrix)

array([[0.        , 0.43731932, 0.45571616, ..., 0.49618922, 0.50591327,
        0.76722777],
       [0.43731932, 0.        , 0.41721333, ..., 0.44581475, 0.48517449,
        0.73476891],
       [0.45571616, 0.41721333, 0.        , ..., 0.45683548, 0.44502755,
        0.72689076],
       ...,
       [0.49618922, 0.44581475, 0.45683548, ..., 0.        , 0.38998164,
        0.74816176],
       [0.50591327, 0.48517449, 0.44502755, ..., 0.38998164, 0.        ,
        0.75157563],
       [0.76722777, 0.73476891, 0.72689076, ..., 0.74816176, 0.75157563,
        0.        ]])

# Consensus clustering

In [18]:
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm

from clustering.ensembles.utils import (
    run_method_and_compute_agreement,
)
from clustering.ensembles.eac import (
    eac_single_coassoc_matrix,
    eac_complete_coassoc_matrix,
    eac_average_coassoc_matrix,
)
from clustering.ensembles.spectral import scc

Define spectral consensus clustering methods with delta values found in pre-analysis:

In [19]:
def scc_020(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.20,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_025(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.25,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_030(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.30,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_050(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.50,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )

In [20]:
all_consensus_methods = set(
    (
        eac_single_coassoc_matrix,
        eac_complete_coassoc_matrix,
        eac_average_coassoc_matrix,
        scc_020,
        scc_025,
        scc_030,
        scc_050,
    )
)
display(all_consensus_methods)

{<function clustering.ensembles.eac.eac_average_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_complete_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_single_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function __main__.scc_020(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_025(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_030(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_050(coassoc_distance_matrix, k, **kwargs)>}

In [21]:
consensus_results = []

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(
            run_method_and_compute_agreement,
            m,
            ensemble_coassoc_matrix,
            full_ensemble,
            k,
            n_init=SC_N_INIT,
            random_state=next(RANDOM_STATES_ITER),
        ): (m.__name__, k)
        for m in all_consensus_methods
        for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1)
    }

    for future in tqdm(as_completed(tasks), total=len(tasks), disable=False, ncols=100):
        method_name, k = tasks[future]
        part, performance_values = future.result()

        method_results = {
            "method": method_name,
            "partition": part,
            "k": k,
        }
        method_results.update(performance_values)

        consensus_results.append(method_results)

100%|█████████████████████████████████████████████████████████████| 413/413 [22:52<00:00,  3.32s/it]


In [22]:
consensus_results = pd.DataFrame(consensus_results)

In [23]:
display(consensus_results.shape)

(413, 12)

In [24]:
consensus_results.head()

,method,partition,k,ari_mean,ari_median,ari_std,ami_mean,ami_median,ami_std,nmi_mean,nmi_median,nmi_std
0,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0.011734,0.000020,0.061126,0.012274,0.000044,0.061146,0.012796,0.000577,0.061114
1,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,0.019360,0.000021,0.082619,0.020361,0.000059,0.082742,0.021393,0.001123,0.082651
2,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,0.024839,0.000017,0.093988,0.026178,0.000070,0.094141,0.027706,0.001662,0.093979
3,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5,0.085062,0.000115,0.244193,0.090376,0.009544,0.232588,0.093898,0.012819,0.231762
4,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6,0.085796,0.000142,0.239327,0.090903,0.009718,0.225980,0.094851,0.013543,0.225064


## Testing

In [25]:
assert not consensus_results.isna().any().any()

In [26]:
# check that the number of clusters in the partitions are the expected ones
_real_k_values = consensus_results["partition"].apply(lambda x: np.unique(x).shape[0])
display(_real_k_values)
assert np.all(consensus_results["k"].values == _real_k_values.values)

0       2
1       3
2       4
3       5
4       6
       ..
408    56
409    57
410    58
411    59
412    60
Name: partition, Length: 413, dtype: int64

## Save

In [27]:
output_file = Path(RESULTS_DIR, "consensus_clustering_runs.pkl").resolve()
display(output_file)

PosixPath('/opt/data/results/clustering/null_sims/shuffle_genes/consensus_clustering/consensus_clustering_runs.pkl')

In [28]:
consensus_results.to_pickle(output_file)